In [17]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process 
import nltk
from nltk.corpus import stopwords
import re

In [2]:
geol = pd.read_csv(r'c:\users\rscott\downloads\SA_LITH_CLIPPED.csv')

In [38]:
Comments_fuzzy =[]
Attr_val_fuzzy =[]
cleanup_dic_list =[]
Litho_dico =[]
Attr_val_Dic =[]
Comments_dic =[]
Comments_fuzzy =[]
First_Filter_list=[]
Comments_dic_tmp = []
#process_list_final =[]
Final_fuzzy =[]
   # make global


Final_split_proc_list =[]
Comm_split_proc_list =[]

#matching
bestmatch=-1
bestlitho=''
top=[]
i=0
attr_val_sub_list=[]


Unpacking https://github.com/RichardScottOZ/dh2loop which is designed to work with a postgres database setup for WAMEX reports
An example using a similar dataset put into wide format  
The dh2loop is parallelised, sensibly when working on millions of records with mulitple 1K length loops  

In [ ]:
#UWA has a database here
##pg_restore -d "testdb" -U pythonsport -f "may2023_data.sql"
Steps to restore the PostgreSQL backup:

1: Create a new database where to load the backup
2: Reload the backup:
	- Using pg_restore on the command line: pg_restore -d "testdb" -U pythonsport -f "may2023_data.sql"
	- Using pgAdmin: Right click on the database where you would like to load the data and select the restore option. Simply indicate the filename and press the restore button.

In [105]:
geol.head()

,PROJECT,SITEID,DEPTHFROM,DEPTHTO,LITHOLOGY1,LITH1_DESCR,LITHOLOGY2,LITH2_DESCR,COMMENTS,clean,...,clean_comment_1,clean_2,clean_comment_2,cleaned_text,cleaned_text_comment,words,words_comment,last,last_comment,scores
0,SARIG,255089_07CDDH0076,112.35,112.68,VC,Vein (Undifferentiated),NaN,NaN,"vein- feldspar, quartz, cordieritefresh coars...",vein,...,vein feldspar quartz cordieritefresh coarse...,[vein],"[vein, feldspar, quartz, cordieritefresh, coar...",vein,vein feldspar quartz cordieritefresh coarse gr...,vein,vein feldspar quartz cordieritefresh coarse gr...,3,73,lithowords scores 0 ...
1,SARIG,214992_MDAC102,6.00,7.00,PORP,Porphyry,NaN,NaN,orange cream green pink to brown quartz porphy...,porphyry,...,orange cream green pink to brown quartz porphy...,[porphyry],"[orange, cream, green, pink, brown, quartz, po...",porphyry,orange cream green pink brown quartz porphyry ...,porphyry,orange cream green pink brown quartz porphyry ...,7,75,lithowords scores 0 ...
2,SARIG,217661_97WDRC14,60.00,61.00,CASI,Calcsilicate Rock,NaN,NaN,Dark grey green black pink garnet psammopelite...,calcsilicate rock,...,dark grey green black pink garnet psammopelite...,"[calcsilicate, rock]","[dark, grey, green, black, pink, garnet, psamm...",calcsilicate rock,dark grey green black pink garnet psammopelite...,calcsilicate rock,dark grey green black pink garnet psammopelite...,16,133,lithowords scores 0 ...
3,SARIG,276137_CHDCU03,406.80,410.40,AUGN,Augen Gneiss,NaN,NaN,stronger magnetite foliations; coarse pink fel...,augen gneiss,...,stronger magnetite foliations coarse pink fel...,"[augen, gneiss]","[stronger, magnetite, foliation, coarse, pink,...",augen gneiss,stronger magnetite foliation coarse pink felds...,augen gneiss,stronger magnetite foliation coarse pink felds...,11,139,lithowords scores 0 ...
4,SARIG,349473_JTRC088,9.00,11.00,CLA,Clay (Undiff. Origin),NaN,NaN,clay,clay,...,clay,[clay],[clay],clay,clay,clay,clay,3,3,lithowords scores 0 ...


In [13]:
thesaurus_geology_comment = pd.read_csv('c:/Users/rscott/OneDrive - OZ Minerals/Exploration2023/dh2loop/thesauri/thesaurus_geology_comment.csv')
thesaurus_geology_lithology = pd.read_csv(r'C:/Users/rscott/OneDrive - OZ Minerals/Exploration2023/dh2loop/thesauri/thesaurus_geology_lithology.csv')
thesaurus_geology_lithology_code = pd.read_csv(r'C:/Users/rscott/OneDrive - OZ Minerals/Exploration2023/dh2loop/thesauri/thesaurus_geology_lithology_code.csv')
thesaurus_geology_hierarchical = pd.read_csv(r'C:/Users/rscott/OneDrive - OZ Minerals/Exploration2023/dh2loop/thesauri/thesaurus_geology_hierarchical.csv')
thesaurus_cleanup = pd.read_csv(r'C:/Users/rscott/OneDrive - OZ Minerals/Exploration2023/dh2loop/thesauri/thesaurus_cleanup.csv')

extra_stopwords = [
    'also',
]
stop = stopwords.words('english') + extra_stopwords

lemma = nltk.WordNetLemmatizer()
    
filename_comm_split = 'Comment_split_list'        

In [97]:
cleanup_dic_list = thesaurus_cleanup['cleanup_term'].tolist()

def clean_text(text):
    '''
    Function clean the text by symbols and specified text, uses cleanup dictionary
    Input: 
         - Text
    output: 
        - Cleaned text

    '''
    text=text.lower().replace('unnamed','').replace('meta','').replace('meta-','').replace('undifferentiated ','').replace('unclassified ','').replace(' undifferentiated ','')
    text=text.replace('differentiated','').replace('undiff','').replace('undiferentiated','').replace('undifferntiates','').replace(' undivided','')
    text=(re.sub('\(.*\)', '', text)) # removes text in parentheses
    text=(re.sub('\[.*\]', '', text)) # removes text in parentheses
    text=text.replace('>','').replace('?','').replace('/',' ')
    text=text.lstrip().rstrip()   #strip of left and right spaces
    text = re.sub('\s+', ' ', text)  # for multiple spaces replace by one space
    text = text.replace('>' , ' ')
    text = text.replace('<', ' ')
    text = text.replace('/', ' ')
    text = text.replace(' \' ', ' ')
    text = text.replace(',', ' ')
    text = text.replace('%', ' ')
    text = text.replace('-', ' ')
    text = text.replace('_', ' ')
    #text = text.replace('', ' ')
    #text = text.replace('+', '')
    text = text.replace('\'', ' ') 
    if text.isnumeric():
        text = re.sub('\d', ' ', text) #replace numbers
    text = text.replace('&' , ' ')
    text = text.replace(',', ' ')
    text = text.replace('.', ' ')
    text = text.replace(':', ' ')
    text = text.replace(';', ' ')
    text = text.replace('$', ' ')
    text = text.replace('@', ' ')
    text = text.replace('\'', '')
    text = text.replace('\\', '')  #replace backslash by space                      
	
    for cleanup_dic_ele in cleanup_dic_list:
        cleaned_item =str(cleanup_dic_ele).replace('(','').replace(')','').replace(',','').replace('\'','')
        text = text.replace('cleaned_item','')
    return text


def tokenize(text, min_len=1):
    '''Function that tokenize a set of strings
    Input:
        -text: set of strings
        -min_len: tokens length
    Output:
        -list containing set of tokens'''

    tokens = [word.lower() for sent in nltk.sent_tokenize(text)
              for word in nltk.word_tokenize(sent)]
    filtered_tokens = []

    for token in tokens:
        if token.isalpha() and len(token) >= min_len:
            filtered_tokens.append(token)

    return [x.lower() for x in filtered_tokens if x not in stop]


def tokenize_and_lemma(text, min_len=0):
    '''Function that retrieves lemmatised tokens
    Inputs:
        -text: set of strings
        -min_len: length of text
    Outputs:
        -list containing lemmatised tokens'''
    filtered_tokens = tokenize(text, min_len=min_len)

    lemmas = [lemma.lemmatize(t) for t in filtered_tokens]
    return lemmas


def joinstr(cleaned_text_1):
    cleaned_text=" ".join(str(x) for x in cleaned_text_1)
    return cleaned_text

def wordsstr(cleaned_text):
    words=(re.sub('\(.*\)', '', cleaned_text)).strip()
    return words

def laststr(words):
    # length of list of words - last one place
    words=words.rstrip('\n\r').split(" ")
    last = len(words)-1
    return last

def lithowords(Litho_dico_ele):
    litho_words = str(Litho_dico_ele).lower().rstrip('\n\r').replace('(','').replace(')','').replace('\'','').replace(',','').split(" ")
    return litho_words

import copy
def makescores(litho, thesaurus_geology_lithology_code):
    #dataframe row and dataframe of thesaurus
    dflc = copy.deepcopy(thesaurus_geology_lithology_code)
    dflc['scores'] = dflc.apply(lambda x: fuzz.token_set_ratio(x.lithowords, litho), axis=1)
    
    return dflc[['lithowords','scores']]
    

In [ ]:
#Comments_dic_tmp
#Attr_val_Dic
#Litho_dico
#cleanup_dict_list

In [90]:
thesaurus_geology_lithology.columns
thesaurus_geology_comment.columns

Index(['attributecolumn', 'lithowords'], dtype='object')

In [59]:
thesaurus_geology_lithology['lithowords'] = thesaurus_geology_lithology['attributecolumn'].apply(lithowords)
thesaurus_geology_lithology_code['lithowords'] = thesaurus_geology_lithology_code['company_lithology'].apply(lithowords)
thesaurus_geology_comment['lithowords'] = thesaurus_geology_lithology['attributecolumn'].apply(lithowords)

In [60]:
thesaurus_geology_lithology_code.head()

,companyid,company_lithocode,company_lithology,lithowords
0,11410,GR,Granite,[granite]
1,11410,MG,Gabbro,[gabbro]
2,11410,SA,Undifferentiated arenite,"[undifferentiated, arenite]"
3,11410,CL,Clay,[clay]
4,12411,Gr,Granitiod undifferentiated,"[granitiod, undifferentiated]"


In [54]:
print("clean 1")
geol['clean_1'] = geol['LITH1_DESCR'].astype('str').apply(clean_text)
geol['clean_comment_1'] = geol['COMMENTS'].astype('str').apply(clean_text)

print("clean 2")
geol['clean_2'] = geol['clean_1'].apply(tokenize_and_lemma)
geol['clean_comment_2'] = geol['clean_comment_1'].apply(tokenize_and_lemma)

print("cleaned text")
geol['cleaned_text'] = geol['clean_2'].apply(joinstr)
geol['cleaned_text_comment'] = geol['clean_comment_2'].apply(joinstr)

print("words str")
geol['words'] = geol['cleaned_text'].apply(wordsstr)
geol['words_comment'] = geol['cleaned_text_comment'].apply(wordsstr)

print("last str")
geol['last'] = geol['words'].apply(laststr)
geol['last_comment'] = geol['words_comment'].apply(laststr)


#cleaned_text_1=clean_text(Attr_val_Dic_ele[2])
#cleaned_text_1=tokenize_and_lemma(cleaned_text_1)
#cleaned_text=" ".join(str(x) for x in cleaned_text_1)  #join each word as string with space


clean 1
clean 2
cleaned text
words str
last str


In [103]:
geol['scores'] = geol.apply(lambda x: makescores(x.cleaned_text, thesaurus_geology_lithology_code), axis=1)

In [ ]:
geol['scores_comment'] = geol.apply(lambda x: makescores(x.cleaned_text_comment, thesaurus_geology_lithology_code), axis=1)

In [57]:
#naive scoring
bestmatch = -1
bestmatch+_comment = -1

for index, row in geol.iterrows():
    top = []
    top_comment = []
    ## all scores dataframe get the max of these for ilth desc
    dfscore = row['scores'].loc[row['scores'].scores >= row['scores'].max()]
    for index, drow in dfscore:
        last = drow['last']
        words = drow['words']
        if drow['scores'] > bestmatch:
            bestmatch = drow['scores']
            bestlitho = drow['lithowords']
            top.append([drow['scores'], drow['lithowords']])
        elif (drow['scores'] ==bestmatch): #equal to previous best match
            if(drow['lithowords']==words[last]): #bonus for being last word in phrase
                bestlitho=drow['lithowords']
                bestmatch=bestmatch*1.01
            else:
                top.append([sc[0],sc[1]])
                top.append([sc[0],sc[1]])
                
    ## all scores dataframe get the max of these for ilth comment
    dfscore_comment = row['scores_comment'].loc[row['scores_comment'].scores >= row['scores_comment'].max()]
    for index, drow in dfscore:
        last = drow['last_comment']
        words = drow['words_comment']
        if drow['scores_comment'] > bestmatch_comment:
            bestmatch_comment = drow['scores_comment']
            bestlitho_comment = drow['lithowords_comment']
            top_comment.append([drow['scores_comment'], drow['lithowords_comment']])
        elif (drow['scores'] ==bestmatch): #equal to previous best match
            if(drow['lithowords_comment']==words[last]): #bonus for being last word in phrase
                bestlitho_comment=drow['lithowords_comment']
                bestmatch_comment=bestmatch_comment*1.01
            else:
                top_comment.append([sc[0],sc[1]])
                top_comment.append([sc[0],sc[1]])


SyntaxError: unexpected EOF while parsing (1077752974.py, line 1)

In [93]:
geol

,PROJECT,SITEID,DEPTHFROM,DEPTHTO,LITHOLOGY1,LITH1_DESCR,LITHOLOGY2,LITH2_DESCR,COMMENTS,clean,...,clean_comment_1,clean_2,clean_comment_2,cleaned_text,cleaned_text_comment,words,words_comment,last,last_comment,scores
0,SARIG,255089_07CDDH0076,112.35,112.68,VC,Vein (Undifferentiated),NaN,NaN,"vein- feldspar, quartz, cordieritefresh coars...",vein,...,vein feldspar quartz cordieritefresh coarse...,[vein],"[vein, feldspar, quartz, cordieritefresh, coar...",vein,vein feldspar quartz cordieritefresh coarse gr...,vein,vein feldspar quartz cordieritefresh coarse gr...,3,73,100
1,SARIG,214992_MDAC102,6.00,7.00,PORP,Porphyry,NaN,NaN,orange cream green pink to brown quartz porphy...,porphyry,...,orange cream green pink to brown quartz porphy...,[porphyry],"[orange, cream, green, pink, brown, quartz, po...",porphyry,orange cream green pink brown quartz porphyry ...,porphyry,orange cream green pink brown quartz porphyry ...,7,75,5
2,SARIG,217661_97WDRC14,60.00,61.00,CASI,Calcsilicate Rock,NaN,NaN,Dark grey green black pink garnet psammopelite...,calcsilicate rock,...,dark grey green black pink garnet psammopelite...,"[calcsilicate, rock]","[dark, grey, green, black, pink, garnet, psamm...",calcsilicate rock,dark grey green black pink garnet psammopelite...,calcsilicate rock,dark grey green black pink garnet psammopelite...,16,133,11
3,SARIG,276137_CHDCU03,406.80,410.40,AUGN,Augen Gneiss,NaN,NaN,stronger magnetite foliations; coarse pink fel...,augen gneiss,...,stronger magnetite foliations coarse pink fel...,"[augen, gneiss]","[stronger, magnetite, foliation, coarse, pink,...",augen gneiss,stronger magnetite foliation coarse pink felds...,augen gneiss,stronger magnetite foliation coarse pink felds...,11,139,9
4,SARIG,349473_JTRC088,9.00,11.00,CLA,Clay (Undiff. Origin),NaN,NaN,clay,clay,...,clay,[clay],[clay],clay,clay,clay,clay,3,3,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365611,SARIG,205905_PH04R083,254.00,270.00,MTXH,Breccia (Undiff. Origin),SSD,Sandstone,"Breccia (Hematite - Quartz Breccia), Hematiti...",breccia,...,breccia hematitic steely grey alteration a...,[breccia],"[breccia, hematitic, steely, grey, alteration,...",breccia,breccia hematitic steely grey alteration hemat...,breccia,breccia hematitic steely grey alteration hemat...,6,185,4
365612,SARIG,255666_06CUD0233,50.97,53.25,MGN,Gneiss,NaN,NaN,gneiss fresh med gr,gneiss,...,gneiss fresh med gr,[gneiss],"[gneiss, fresh, med, gr]",gneiss,gneiss fresh med gr,gneiss,gneiss fresh med gr,5,18,5
365613,SARIG,298834_MAR0879,21.00,22.50,SST,Sand,NaN,NaN,Sand.Grain size: Fine-Very coarse.Sorting: Poo...,sand,...,sand grain size fine very coarse sorting poo...,[sand],"[sand, grain, size, fine, coarse, sorting, poo...",sand,sand grain size fine coarse sorting poor colou...,sand,sand grain size fine coarse sorting poor colou...,3,59,3
365614,SARIG,151983_HKN 23,40.00,42.00,BIFM,Banded Iron Formation,NaN,NaN,BIF as above . Water table.,banded iron formation,...,bif as above water table,"[banded, iron, formation]","[bif, water, table]",banded iron formation,bif water table,banded iron formation,bif water table,20,14,13


In [46]:
fuzz.token_set_ratio

<function fuzzywuzzy.fuzz.token_set_ratio(s1, s2, force_ascii=True, full_process=True)>

In [63]:
geol.describe()

,DEPTHFROM,DEPTHTO,last,last_comment,scores
count,365616.000000,365616.000000,365616.000000,365616.000000,365616.000000
mean,128.977299,134.267008,6.607189,64.784561,96.339794
std,264.945542,267.056535,5.426168,55.705698,16.589045
min,0.000000,0.010000,2.000000,-1.000000,0.000000
25%,11.000000,15.000000,3.000000,27.000000,100.000000
50%,33.000000,37.500000,5.000000,54.000000,100.000000
75%,92.000000,98.000000,8.000000,83.000000,100.000000
max,2437.600000,2459.000000,54.000000,944.000000,100.000000


In [64]:
geol.loc[geol.scores < 100]

,PROJECT,SITEID,DEPTHFROM,DEPTHTO,LITHOLOGY1,LITH1_DESCR,LITHOLOGY2,LITH2_DESCR,COMMENTS,clean,...,clean_comment_1,clean_2,clean_comment_2,cleaned_text,cleaned_text_comment,words,words_comment,last,last_comment,scores
3,SARIG,276137_CHDCU03,406.80,410.40,AUGN,Augen Gneiss,NaN,NaN,stronger magnetite foliations; coarse pink fel...,augen gneiss,...,stronger magnetite foliations coarse pink fel...,"[augen, gneiss]","[stronger, magnetite, foliation, coarse, pink,...",augen gneiss,stronger magnetite foliation coarse pink felds...,augen gneiss,stronger magnetite foliation coarse pink felds...,11,139,67
27,SARIG,20169_PR 605,47.24,56.39,NaN,NaN,NaN,NaN,red blue ore,nan,...,red blue ore,[nan],"[red, blue, ore]",nan,red blue ore,nan,red blue ore,2,11,0
33,SARIG,268901_LEAC174,48.00,58.00,AUGN,Augen Gneiss,NaN,NaN,Tunkillia Augen Gneiss,augen gneiss,...,tunkillia augen gneiss,"[augen, gneiss]","[tunkillia, augen, gneiss]",augen gneiss,tunkillia augen gneiss,augen gneiss,tunkillia augen gneiss,11,21,67
35,SARIG,13477_MINES DEPT WA 25,25.60,31.09,NaN,NaN,NaN,NaN,"No cuttings return, probably sand as above",nan,...,no cuttings return probably sand as above,[nan],"[cutting, return, probably, sand]",nan,cutting return probably sand,nan,cutting return probably sand,2,27,0
80,SARIG,202476_NHRAB 157,19.00,22.00,CLYW,NaN,NaN,NaN,"green clay with grey qtz chips, sericite-epido...",nan,...,green clay with grey qtz chips sericite epido...,[nan],"[green, clay, grey, qtz, chip, sericite, epido...",nan,green clay grey qtz chip sericite epidote kfel...,nan,green clay grey qtz chip sericite epidote kfel...,2,79,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365400,SARIG,210780_RMGLRC001,225.00,250.00,VRD,Rhyodacite,NaN,NaN,Rhyodacite.,rhyodacite,...,rhyodacite,[rhyodacite],[rhyodacite],rhyodacite,rhyodacite,rhyodacite,rhyodacite,9,9,0
365475,SARIG,5960_304011101,4.27,5.49,NaN,NaN,NaN,NaN,Aeolianite as above sub silty moderately cemen...,nan,...,aeolianite as above sub silty moderately cemen...,[nan],"[aeolianite, sub, silty, moderately, cemented,...",nan,aeolianite sub silty moderately cemented pale ...,nan,aeolianite sub silty moderately cemented pale ...,2,58,0
365511,SARIG,357_MURRAWIDGINEE BORE & WELL,156.36,156.97,NaN,NaN,NaN,NaN,black rock,nan,...,black rock,[nan],"[black, rock]",nan,black rock,nan,black rock,2,9,0
365545,SARIG,252739_GOMA DH4,482.80,487.40,OTGN,Orthogneiss,GRAN,Granite,med to coarse grained foliated felsic. Contain...,orthogneiss,...,med to coarse grained foliated felsic contain...,[orthogneiss],"[med, coarse, grained, foliated, felsic, conta...",orthogneiss,med coarse grained foliated felsic contains pr...,orthogneiss,med coarse grained foliated felsic contains pr...,10,112,0


In [74]:
fuzz.token_set_ratio(geol.iloc[3]['cleaned_text'],thesaurus_geology_lithology_code.iloc[0]['lithowords'])

32

In [76]:
print(geol.iloc[3]['cleaned_text'],thesaurus_geology_lithology_code.iloc[0]['lithowords'])

augen gneiss ['granite']


In [73]:
type(thesaurus_geology_lithology_code.iloc[0]['lithowords'])

list

In [81]:
base = -1
for index, row in thesaurus_geology_lithology_code.iterrows():
    s = fuzz.token_set_ratio(geol.iloc[3]['cleaned_text'],row['lithowords'])
    
    if s > base:
        base = s
    print(s,row['lithowords'],base )
        
    

32 ['granite'] 32
11 ['gabbro'] 32
39 ['undifferentiated', 'arenite'] 39
12 ['clay'] 39
32 ['granitiod', 'undifferentiated'] 39
22 ['schists', '[undifferentiated]'] 39
32 ['andesite', '[undifferentiated]'] 39
32 ['granite'] 39
24 ['silts', '[undifferentiated]'] 39
24 ['colluvium', '-', 'local', 'coarse'] 39
25 ['talc-chlorite', 'schist'] 39
30 ['high', 'mag', 'basalt'] 39
20 ['dolerite'] 39
26 ['sedimentary', 'rocks-undivided'] 39
11 ['gabbro'] 39
12 ['chert'] 39
33 ['mafic', 'undiff'] 39
10 ['dolomite'] 39
24 ['shale'] 39
14 ['ultramafic', 'schist'] 39
29 ['no', 'sample-return'] 39
28 ['ultramafic', 'undiff'] 39
21 ['ultramafic', 'rocks-undivided'] 39
37 ['altered', 'beyond', 'recognition'] 39
24 ['altered', 'rock', 'unknown', 'precursor'] 39
13 ['bif'] 39
27 ['banded', 'iron', 'formation', 'bif'] 39
22 ['schist'] 39
37 ['sediment', 'undiff'] 39
19 ['siltstone'] 39
38 ['sandstone'] 39
32 ['not', 'logged\\recorded'] 39
30 ['mafic-undivided'] 39
33 ['mottled', 'zone'] 39
30 ['banded', '

In [98]:
makescores(geol.iloc[3]['cleaned_text'],thesaurus_geology_lithology_code)

,lithowords,scores
0,[granite],32
1,[gabbro],11
2,"[undifferentiated, arenite]",39
3,[clay],12
4,"[granitiod, undifferentiated]",32
...,...,...
1547,[dunite],33
1548,"[olivine, rich]",25
1549,[peridotites],26
1550,[pyroxenites],35


In [94]:
geoltest = geol[0:2]
geoltest

,PROJECT,SITEID,DEPTHFROM,DEPTHTO,LITHOLOGY1,LITH1_DESCR,LITHOLOGY2,LITH2_DESCR,COMMENTS,clean,...,clean_comment_1,clean_2,clean_comment_2,cleaned_text,cleaned_text_comment,words,words_comment,last,last_comment,scores
0,SARIG,255089_07CDDH0076,112.35,112.68,VC,Vein (Undifferentiated),NaN,NaN,"vein- feldspar, quartz, cordieritefresh coars...",vein,...,vein feldspar quartz cordieritefresh coarse...,[vein],"[vein, feldspar, quartz, cordieritefresh, coar...",vein,vein feldspar quartz cordieritefresh coarse gr...,vein,vein feldspar quartz cordieritefresh coarse gr...,3,73,100
1,SARIG,214992_MDAC102,6.00,7.00,PORP,Porphyry,NaN,NaN,orange cream green pink to brown quartz porphy...,porphyry,...,orange cream green pink to brown quartz porphy...,[porphyry],"[orange, cream, green, pink, brown, quartz, po...",porphyry,orange cream green pink brown quartz porphyry ...,porphyry,orange cream green pink brown quartz porphyry ...,7,75,5


In [99]:
geoltest['scores'] = geoltest.apply(lambda x: makescores(x.cleaned_text, thesaurus_geology_lithology_code), axis=1)

C:\Users\rscott\AppData\Local\Temp\ipykernel_16688\113751796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geoltest['scores'] = geoltest.apply(lambda x: makescores(x.cleaned_text, thesaurus_geology_lithology_code), axis=1)


In [102]:
geoltest['scores'].values

array([                         lithowords  scores
       0                         [granite]      18
       1                          [gabbro]       0
       2       [undifferentiated, arenite]      21
       3                            [clay]       0
       4     [granitiod, undifferentiated]      13
       ...                             ...     ...
       1547                       [dunite]      20
       1548                [olivine, rich]      38
       1549                  [peridotites]      27
       1550                  [pyroxenites]      27
       1551                         [vein]     100

       [1552 rows x 2 columns]                    ,
                                lithowords  scores
       0                         [granite]      13
       1                          [gabbro]      14
       2       [undifferentiated, arenite]      12
       3                            [clay]      17
       4     [granitiod, undifferentiated]      12
       ...                   